In [92]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re
from datetime import date, timedelta

In [76]:
def extract_from_url(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")
    return soup

def remove_html_tags(text):
    return re.sub("<.*?>", "", text)

In [95]:
start_date = date(2021, 1, 1)
end_date = date(2021, 2, 1)

In [111]:
articles_for_page

[]

In [114]:
dates = list(pd.date_range(start_date, end_date))

all_articles = []

for date in dates:
    date = date.strftime("%Y-%m-%d")
    date_str = f"&date={date}"
    print(date_str)
    
    counter = 1
    has_next_page = True
    
    while has_next_page:
        if counter == 1:
            page_str = ""
        else:
            page_str = f"&page={counter}"
        
        soup = extract_from_url(f"https://indeks.kompas.com/?site=all{date_str}{page_str}")
        articles_for_page = soup.findAll("div", {"class": "article__list clearfix"})
        if not articles_for_page:
            has_next_page = False
        
        for article in articles_for_page:
            link = article.findAll("a", {"class": "article__link"})[0].get("href")
            text = extract_from_url(link).findAll("div", {"class": "read__content"})[0]
            text = remove_html_tags(str(text))
            title = article.findAll("h3", {"class": "article__title"})[0].get_text()
            topic = article.findAll("div", {"class": "article__subtitle"})[0].get_text()
            datetime = article.findAll("div", {"class": "article__date"})[0].get_text()
            all_articles.append({"link": link,
                                 "title": title,
                                 "topic": topic,
                                 "datetime": datetime,
                                 "text": text})
        counter += 1
    continue

&date=2021-01-01
&date=2021-01-02
&date=2021-01-03
&date=2021-01-04
&date=2021-01-05
&date=2021-01-06
&date=2021-01-07


IndexError: list index out of range

In [80]:
df = pd.DataFrame(all_articles)